# Neighborhood

<!-- SUMMARY: Definition of the Neighborhood concept. This tutorial gives hints on the parameters of the Moving Neighborhood -->

<!-- CATEGORY: Basic_Objects -->

This file is meant to demonstrate the use of gstlearn for Moving Neighborhood search

In [ ]:
import numpy as np
import pandas as pd
import sys
import os
import gstlearn as gl
import gstlearn.plot as gp
import gstlearn.document as gdoc
import matplotlib.pyplot as plt
import random as rnd

gdoc.setNoScroll()

Setting some global variables

In [ ]:
# Set the Global Options
verbose = True
flagGraphic = True

# Define the Space Dimension
ndim = 2
gl.defineDefaultSpace(gl.ESpaceType.RN, ndim)

# Set the Seed for the Random Number generator
gl.law_set_random_seed(5584)
rnd.seed(13155)

In this paragraph, we generate a Poisson data set and check various neighborhoods around one specific node of a regular grid.

In [ ]:
dxref = 0.1
grid = gl.DbGrid.create(nx=[10,10],dx=[dxref,dxref])
xlim = grid.getExtrema(0)
ylim = grid.getExtrema(1)

# Poisson Data Set

In [ ]:
coormin = grid.getCoorMinimum()
coormax = grid.getCoorMaximum()
nech = 100
data = gl.Db.createFromBox(nech, coormin, coormax)

In [ ]:
gp.setDefaultGeographic(xlim=xlim, ylim=ylim, dims=[7,7])
ax = data.plot()

## Checking standard neighborhood

Defining a standard Moving Neighborhood

In [ ]:
nmini = 1
nmaxi = 15
radius = 0.3
nsect = 8
nsmax = 3
neigh = gl.NeighMoving.create(False, nmaxi, radius, nmini, nsect, nsmax)
neigh

Checking the neighborhood around a central grid node

In [ ]:
node = 55
neigh.attach(data, grid)
ranks = neigh.select(node)

dataSel = data.clone()
dum = dataSel.addSelectionByRanks(ranks)

In [ ]:
ax = data.plot()
ax = dataSel.plot(color='blue')
ax.neigh(neigh, grid, node, flagCell=True)
ax.decoration("Standard Neighborhood")

## Defining variable block extensions

In this section, we will generate variables in the Grid which contain the cell extension: this cell extension replaces the constant value of the mesh.

In [ ]:
nech = grid.getSampleNumber()
mini = 0.5
maxi = 2.5
blx = []
bly = []
for i in range(nech):
    blx.append(dxref * rnd.uniform(mini, maxi))
    bly.append(dxref * rnd.uniform(mini, maxi))

dum = grid.addColumns(blx, "X-ext", gl.ELoc.BLEX, 0)
dum = grid.addColumns(bly, "Y-ext", gl.ELoc.BLEX, 1)

The following display shows each block with its center and its cell extension.

In [ ]:
ax = data.plot()
ax = dataSel.plot(color='blue')
ax.decoration(title="Variable Block Size")

for node in range(nech):
    ax = gp.sample(grid.getSampleCoordinates(node))
    ax = gp.curve(grid.getCellEdges(node))

We choose a specific cell again and determine the standard neighrbohood

In [ ]:
node = 56
neigh.attach(data, grid)
ranks = neigh.select(node)

dataSel = data.clone()
dum = dataSel.addSelectionByRanks(ranks)

In [ ]:
ax = data.plot()
ax = dataSel.plot(color='blue')
ax.neigh(neigh, grid, node, flagCell=True)
ax.decoration(title="Standard Neighborhood")

Use the Cell neighborhood to force the selection of all samples belonging to the block

In [ ]:
nmini = 1
neigh = gl.NeighCell.create(False, nmini)
neigh

In [ ]:
node = 56
neigh.attach(data, grid)
ranks = neigh.select(node)

dataSel = data.clone()
dum = dataSel.addSelectionByRanks(ranks)

In [ ]:
ax = data.plot()
ax = dataSel.plot(color='blue')
ax.neigh(neigh, grid, node, flagCell=True)
ax.decoration(title="Neighborhood forced to the Cell")